# Simulating voting rules

## Introduction

Voting rules can be confusing. To illustrate the options as discussed in the lectures, we simulate the voting rules and the effects they can have. In this demo, "agent" and "voter" is used interchangably.

In [21]:
import random
import numpy as np
import pandas as pd
import string
from collections import Counter

## Model definition

### Defining functions

We first define the functions `create_voters` and `create_alternatives` to create specific sets $N$ and $A$, respectively.

In [22]:
def create_voters(n_voters: int = 0) -> list[int]:
    """
    Create a list of voters of size `n_voters`.
    If `n_voters` is equal to 0, a random number of voters in the range 
    [3, 10] will be created.
    """

    if n_voters is None:
        n_voters = random.randint(3, 10)

    return [i for i in range(n_voters)]


def create_alternatives(n_alternatives: int = 0) -> list[str]:
    """
    Create a list of alternatives of size `n_alternatives`.
    If `n_alternatives` is equal to 0, a random number of alternatives in the range 
    [3, 10] will be created.
    """

    if n_alternatives is None:
        n_alternatives = random.randint(3, 10)

    return [string.ascii_letters[i] for i in range(n_alternatives)]


print(f'{create_voters(5)=}')
print(f'{create_alternatives(5)=}')

create_voters(5)=[0, 1, 2, 3, 4]
create_alternatives(5)=['a', 'b', 'c', 'd', 'e']


We define a function `create_preference_profile` for creating preference profiles. For each agent $i \in N$ it creates a preference $\succ_i$ containing alternatives from the set $A$:

In [23]:
def create_preference_profile(voters: list[int], alternatives: list[str]) -> dict:
    """
    Create a preference profile for a given number of voters and alternatives.
    Randomized on each call.
    """
    
    preference_profile = {}

    for voter in voters:
        preference_profile[voter] = ''.join(random.sample(alternatives, k=len(alternatives)))
        
    return preference_profile

Functions to get a voter's preference and a voter's top choice for some preference:

In [24]:
def voter_preference(profile: dict, voter: int) -> str:
    """
    Returns the preference of a voter.
    """
    
    return profile[voter]

def top_choice(profile: dict, voter: int) -> str:
    """
    Returns the top choice of a voter.
    """
    
    return profile[voter][0]

## Description of a sample preference profile

In [25]:
# sample preference profile for 4 voters and 3 alternatives
voters, alternatives = create_voters(4), create_alternatives(3)
profile = create_preference_profile(voters, alternatives)
print('Sample preference profile:')
print(f'{profile=}')


print('\nPreferences by voter:')

for voter in voters:
    print(f'Voter {voter}: {voter_preference(profile, voter)}')

preference_count = {}
for preference in profile.values():
    preference_count[preference] = preference_count.get(preference, 0) + 1


print('\nPreferences by frequency:')

for preference, count in sorted(preference_count.items(), key=lambda x: x[1], reverse=True):
    print(f'{preference}: {preference_count[preference]}')

Sample preference profile:
profile={0: 'abc', 1: 'abc', 2: 'acb', 3: 'acb'}

Preferences by voter:
Voter 0: abc
Voter 1: abc
Voter 2: acb
Voter 3: acb

Preferences by frequency:
abc: 2
acb: 2


## The dictatorship rule $F_{DCT}$

In [26]:
def dictatorship_rule(profile: dict, voter: int) -> str:
    """
    This rule returns the top choice of a voter.
    """
    
    return top_choice(profile, voter)

In [77]:
n_voters = 4
n_alternatives = 3
print(f'{n_voters=}')
print(f'{n_alternatives=}')

voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_preference_profile(voters, alternatives)
print(f'{profile=}')

dictator = random.choice(voters)

print('\nDictatorship rule:')
print(f'{dictator=}')
print(f'Result: {dictatorship_rule(profile, dictator)}')

n_voters=4
n_alternatives=3
profile={0: 'bca', 1: 'acb', 2: 'acb', 3: 'cab'}

Dictatorship rule:
dictator=2
Result: a


## The majority rule $F_{MAJ}$

The majority rule does not always have an outcome. In that case, the function `majority_rule` returns `None`. This can happen when `n_voters` is even and/or `n_alternatives` is greater than 2.

In [82]:
def majority_rule(profile: dict) -> str | None:
    """
    This rule returns the majority top choice.
    If there is no majority, it returns None.
    """
    
    majority = len(profile) // 2 + 1

    top_choices = [top_choice(profile, voter) for voter in profile]
    top_choice_count = Counter(top_choices)

    for top, count in top_choice_count.items():
        if count >= majority:
            return top
        
    return None

In [93]:
n_voters = 3
n_alternatives = 2
print(f'{n_voters=}')
print(f'{n_alternatives=}')

voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_preference_profile(voters, alternatives)
print(f'{profile=}')

print('\nMajority rule:')

result = majority_rule(profile)
print(f'Result: {result}')

n_voters=3
n_alternatives=2
profile={0: 'ab', 1: 'ab', 2: 'ab'}

Majority rule:
Result: a


## The plurarity rule $F_{PLU}$

In [94]:
...


Ellipsis

## Duverger's law

In [95]:
...

Ellipsis

## The plurarity with runoff rule $F_{RUN}$

In [96]:
...

Ellipsis

## The Condorcet rule $F_{CON}$

In [98]:
...

Ellipsis

## The Borda rule $F_{BOR}$

In [99]:
...

Ellipsis

## Various scoring rules

In [100]:
...

Ellipsis